In [16]:
import twitter
import csv
import json
import re
import requests
import lxml.html as html
from urllib2 import urlopen
import pandas as pd

In [149]:
class TJLoader:

    def __init__(self):
        self._news_pages = "https://tjournal.ru/paper/page/{}"
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        for i in range(count):
            print i+min_index
            page = html.parse(urlopen(self._news_pages.format(i+min_index)))
            divs = page.getroot().find_class('b-articles__b__title')

            for div in divs:
                links.append(div.getchildren()[1].get("href"))

        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с tjournal, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__title")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("b-article__infoline__comments")
        comment = int(comments[0].find("b").text.replace(" ", ""))

        # Теги
        tags = root.find_class("b-article__tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infoline__date")
        date = self._parse_date(date[0].text)
        

        return {"title": title, "views": view, "comments": comment, "tags": tag_list, "date": date}


    def get_tj_news_info(self, min_index=1, count=30, last_news=""):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :last_news: str, id последней новости, которую нужно скачать
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        for link in links:
            link_info = self.get_link_info(link)
            link_id = link.split("/")[-1]
            # Если добрались до последней новости, которую хотим сохранить
            print link_id, last_news
            if (link_id == last_news):
                break
            link_info["id"] = link_id
            link_info_list.append(link_info)

        return link_info_list

In [150]:
loader = TJLoader()
pages = loader.get_tj_news_info(min_index=1, count=1)

1
windows-30 
colombian-isis 
icloud-thief 
breaking-mad-cientouno 
lawrence-removed 
rkn-wiki-narcotics-science 
good-dinosaur-review 
obeschania-weekly-59 
three-suns 
coub-boom 
natgeo-best-russian-region 
watch-paint-dry 
putin-washington 
twitter-cat-fact-spam-snowden 
nfc-cardsmobile 
state-of-cyberwarfare 
tinder-pr-fail 
parking-fire 
truckers-riot 
ballistics-calculator 
zano-drone 
twitch-madison-whitera 
anonymous-vs-isis 
panasonic-magic-megaphone 
grugq-telegram 
usembru-vs-izvestia 
5-reasons-of-metal 
samsung-back-to-the-future 
rkn-wiki-narcotics 
cat-cucumbers 


In [151]:
pd.DataFrame(pages)

,comments,date,id,tags,title,views
0,69,2015-11-21 19:18,windows-30,"[Apple, Microsoft, Windows, Mac OS X, Билл Гей...",Ретро: 30 лет Windows,8205
1,11,2015-11-21 16:19,colombian-isis,"[фейки, ИГИЛ, РЕН ТВ, колумбийские мафиози]","Фейк: Колумбийские мафиози, убившие террористо...",9995
2,168,2015-11-20 22:35,icloud-thief,"[iCloud, мошенники, блокировка айфона, угон ай...",«Наивность жертв определяется специальным скри...,43355
3,4,2015-11-20 18:25,breaking-mad-cientouno,"[Breaking Mad, преступность, животные, драмати...",Драматичные новости недели от Breaking Mad: ст...,4082
4,60,2015-11-20 17:58,lawrence-removed,"[Дженнифер Лоуренс, Голодные игры, Израиль, Го...",Изображение Дженнифер Лоуренс убрали с части и...,17393
5,29,2015-11-20 17:09,rkn-wiki-narcotics-science,"[Роскомнадзор, блокировки, наука, Википедия, В...",«Википедия» поспорила с Роскомнадзором о научн...,4990
6,6,2015-11-20 15:48,good-dinosaur-review,"[Disney, обзоры, рецензии, Pixar, Хороший дино...","Хороший, плохой, злой",6345
7,14,2015-11-20 13:12,obeschania-weekly-59,"[Обещания.Ru, дайджест обещаний за неделю, Исл...",Обещания недели: санкционный сыр от Деда Мороз...,3095
8,27,2015-11-20 12:46,three-suns,"[фото, социальные сети, Instagram, Челябинск, ...",Фото: «Три солнца» над Челябинском,8235
9,14,2015-11-20 12:09,coub-boom,"[видео, музыка, Coub, коубирация, iBenji, Вени...",Коубирация: Бесконечный бум,4463


In [121]:
import twitter
from datetime import datetime, timedelta
from dateutil import tz
import time

class TwitterLoader:


    def __init__(self):
        CONSUMER_KEY = 'BOuuaMDhNhm6yx0rzqK8bMsbI'
        CONSUMER_SECRET = '3DybJwlkXd2vU6R385yLA8yJblYJltLtwojySD9AVs04ShauZ0'

        ACCESS_TOKEN_KEY = '3712177576-of3jzZ8gNmlPDfPjPyR0Ljw1Ao2IXdTqX9dZGDZ'
        ACCESS_TOKEN_SECRET = 'Ky7iKwByHNXX3UMfuMhv6UgVx2IhjLo3KmwpsBQz35wtG'

        self.api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}

        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"

        self._news_uri = "https://tjournal.ru/p/"



    def _parse_date(self, date):

        """

        :param date: str, дата в формате твиттера - "Sat Nov 21 17:00:29 +0000 2015"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        date_array = date.split(' ')
        month = self._month_dict[date_array[1]]
        day = int(date_array[2])
        time = date_array[3]
        year = date_array[5]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
        moscow_date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        return str(moscow_date).split("+")[0]



    def loadTweetWithLink(self, link, date):
        """

        :param link: str, ключевое слово для поиска, вданном случае - ссылка на письмо
        :param date: str, дата, до которой искать твиты
        :return: список словариков с информацией о УНИКАЛЬНЫХ твитах по запросу link
        """
        set_id = set()
        tweet_list = []
        result = self.api.GetSearch(term=link, until=date, count=100000)

        for res in result:

            tw_id = res.GetId()
            # Если мы уже обрабатывали этот твит - идем дальше
            if tw_id in set_id:
                continue

            retweeted_status = res.GetRetweeted_status()
            # Если это не ретвит
            if retweeted_status is None:
                is_retweet = 0
                retweeted_count = res.GetRetweetCount()
                favorite_count = res.GetFavoriteCount()
            else:
                is_retweet = 1
                retweeted_count = 0
                favorite_count = 0
            set_id.add(tw_id)

            created_at = res.GetCreatedAt()
            created_at = self._parse_date(created_at)

            # Данные о пользователе
            user = res.GetUser()
            followers_count = user.followers_count
            listed_count = user.listed_count
            friends_count = user.friends_count
            favourites_count = user.favourites_count
            statuses_count = user.statuses_count


            link = link.split("/")[-1]

            tw_dict= {
                    "id": link,
                    "tw_id":tw_id,
                    "retweeted_count": retweeted_count,
                    "favorite_count":favorite_count,
                    "is_retweet": is_retweet,
                    "created_at":created_at,
                    "user_followers_count":followers_count,
                    "user_listed_count": listed_count,
                    "user_friends_count":friends_count,
                    "user_favourites_count":favourites_count,
                    "user_statuses_count":statuses_count
                    }

            tweet_list.append(tw_dict)

        return tweet_list

    def _get_next_date(self, date, days):
        """

        :param date: str, дата
        :param days: int, количество дней
        :return: возвращает дату через days-дней после date
        """
        date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        date += timedelta(days=days)
        return str(date).split(' ')[0]


    def load_tweets_by_term(self, news_list, days_after_news=2):
        """

        :param news_list: list(str), список словариков с информацией о новости
        :param days_after_news:  int, количество дней после публикации новости, до которой искать
        """
        result_list = []
        for news in news_list:
            tweets = []
            try:
                uri = self._news_uri + news["id"]
                until_date = self._get_next_date(news["date"], days_after_news)
                tweets = self.loadTweetWithLink(uri, until_date)
            except twitter.error.TwitterError as ex:
                print str(ex)
                if str(ex) == self._RATE_LIMIT:
                    sleep_time = self.api.GetSleepTime("statuses/user_timeline")
                    print "Спим {} сек.".format(sleep_time)
                    time.sleep(sleep_time+2)
                    tweets = self.loadTweetWithLink(news)

            result_list+= tweets
            
        return result_list

In [122]:
tw = TwitterLoader()

In [110]:
twds = tw.loadTweetWithLink("https://tjournal.ru/p/windows-30", "2015-11-22")

In [123]:
twi_data = tw.load_tweets_by_term(pages)

In [124]:
pd.DataFrame(twi_data)

,created_at,favorite_count,id,is_retweet,retweeted_count,tw_id,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,2015-11-21 16:21:50,16,colombian-isis,0,26,668056728957788162,132,241272,675,1157,39796
1,2015-11-22 01:21:09,0,colombian-isis,1,0,668192452646424576,28,168,717,7,7321
2,2015-11-21 22:49:56,0,colombian-isis,0,0,668154396484702209,1,240,183,6,55897
3,2015-11-21 19:43:15,0,colombian-isis,1,0,668107415318093825,152,40,127,0,283
4,2015-11-21 19:39:54,0,colombian-isis,1,0,668106572606914560,136,116,49,3,7054
5,2015-11-21 19:06:01,0,colombian-isis,0,0,668098048086683648,17538,2045,387,85,126402
6,2015-11-21 18:37:00,0,colombian-isis,0,0,668090743161008128,85,74,93,2,3026
7,2015-11-21 18:23:59,0,colombian-isis,1,0,668087467514462208,741,136,339,1,20839
8,2015-11-21 18:06:19,0,colombian-isis,1,0,668083024005844992,5569,271,263,9,78648
9,2015-11-21 17:52:34,0,colombian-isis,1,0,668079563281408000,20,44,126,4,4541


In [120]:
twi_data

[[{'created_at': '2015-11-21 16:21:50',
   'favorite_count': 16,
   'id': 'colombian-isis',
   'retweeted_count': 26,
   'tw_id': 668056728957788162,
   'user_favourites_count': 132,
   'user_followers_count': 241270,
   'user_friends_count': 675,
   'user_listed_count': 1157,
   'user_statuses_count': 39796},
  {'created_at': '2015-11-22 01:21:09',
   'favorite_count': 0,
   'id': 'colombian-isis',
   'retweeted_count': 0,
   'tw_id': 668192452646424576,
   'user_favourites_count': 28,
   'user_followers_count': 168,
   'user_friends_count': 717,
   'user_listed_count': 7,
   'user_statuses_count': 7321},
  {'created_at': '2015-11-21 22:49:56',
   'favorite_count': 0,
   'id': 'colombian-isis',
   'retweeted_count': 0,
   'tw_id': 668154396484702209,
   'user_favourites_count': 1,
   'user_followers_count': 240,
   'user_friends_count': 183,
   'user_listed_count': 6,
   'user_statuses_count': 55893},
  {'created_at': '2015-11-21 19:43:15',
   'favorite_count': 0,
   'id': 'colombian-

In [146]:
a=loader.get_tj_news_info(min_index=2, count=1, last_news="hunger-games-final-review")

google-maps-body-found hunger-games-final-review
fb-safety-check-for-everybody hunger-games-final-review
blue-light-no-sleep hunger-games-final-review
no-converge-for-ipad-and-macbook hunger-games-final-review
prince-charles-bottom hunger-games-final-review
orly-airport-windows-311 hunger-games-final-review
android-chrome-pacsec hunger-games-final-review
google-self-driving-car-and-police hunger-games-final-review
survival-on-billboard hunger-games-final-review
pleer-com-lastwords hunger-games-final-review
vending-for-cars hunger-games-final-review
donates-hate-tweet hunger-games-final-review
tor-carnegie-mellon hunger-games-final-review
vk-a321-merchandise hunger-games-final-review
apple-icloud-fishing-revenge hunger-games-final-review
microsoft-emotion-api hunger-games-final-review
petrenko-charlie-hebdo-mock hunger-games-final-review
moscow-fog hunger-games-final-review
vk-actual-news hunger-games-final-review
instaagent-instagram-app hunger-games-final-review


KeyboardInterrupt: 

In [145]:
pd.DataFrame(a)

,comments,date,id,tags,title,views
0,32,2015-11-16 15:31,google-maps-body-found,"[Google, США, Google Maps, карты, поиск пропав...",Утонувшую машину пропавшего в 2006 году мужчин...,16491
1,21,2015-11-16 13:26,fb-safety-check-for-everybody,"[Facebook, Марк Цукерберг, стрельба в Париже, ...",Facebook переосмыслил свои «оповещения о безоп...,13325
2,59,2015-11-16 12:56,blue-light-no-sleep,"[смартфоны перед сном, экраны смартфонов]",Известный британский врач заявил о необходимос...,9841
3,37,2015-11-16 11:59,no-converge-for-ipad-and-macbook,"[Apple, Microsoft, Тим Кук, iPad, iOS, Mac, iP...",Тим Кук объяснил различиями между Mac и PC неж...,6910
4,20,2015-11-13 23:21,prince-charles-bottom,"[австралийские СМИ, британские СМИ, Джанин Кир...",«Слишком низко опустившаяся» женская рука на с...,18592
5,39,2015-11-13 19:47,orly-airport-windows-311,"[Microsoft, Windows, аэропорты, аэропорт Париж...",Парижский аэропорт пережил сбой систем безопас...,12145
6,25,2015-11-13 16:18,android-chrome-pacsec,"[Android, уязвимости, вирусы, Javascript, эксп...",Уязвимость Chrome на Android позволила взломат...,8070
7,48,2015-11-13 10:23,google-self-driving-car-and-police,"[Google, автомобили, беспилотные автомобили, с...",Самоуправляемый автомобиль Google попытались о...,7867
8,49,2015-11-12 18:29,survival-on-billboard,"[Microsoft, Rise of the Tomb Raider, прямой эф...",Microsoft позволила пользователям сети выпуска...,27347
9,34,2015-11-12 17:02,pleer-com-lastwords,"[ВКонтакте, пиратство, копирайт, антипиратский...",Pleer.com попросил государство «не наводить по...,4676
